### Arima Forecasting

In [24]:
import boto3
import sagemaker
import csv
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.amazon.amazon_estimator import get_image_uri

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [35]:
!aws s3 cp --recursive s3://aurelia-resort-data/model_train/data_csv/ ./data

download: s3://aurelia-resort-data/model_train/data_csv/test.csv to data/test.csv
download: s3://aurelia-resort-data/model_train/data_csv/test_nans.csv to data/test_nans.csv
download: s3://aurelia-resort-data/model_train/data_csv/train_nans.csv to data/train_nans.csv
download: s3://aurelia-resort-data/model_train/data_csv/train.csv to data/train.csv


In [36]:
train = pd.read_csv('data/train.csv',  header=None)
test = pd.read_csv('data/test.csv', header=None)

In [38]:
from sagemaker import image_uris

linearlearner_image_uri = sagemaker.image_uris.retrieve("linear-learner", region="us-east-1", version="latest")

print(linearlearner_image_uri)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


In [39]:
bucket_name = 'aurelia-resort-data'
s3_prefix = 'linear_learner'

In [47]:
container = get_image_uri(region_name=region, 
                          repo_name='linear-learner', 
                          repo_version='latest')

linear_estimator = sagemaker.estimator.Estimator(container,
                                              role=sagemaker.get_execution_role(),
                                              instance_count=1,
                                              instance_type='ml.m4.xlarge',
                                              output_path=f's3://aurelia-resort-data/model_train/linear-learner//output',
                                              sagemaker_session=sagemaker_session,)

linear_estimator.set_hyperparameters(predictor_type = 'regressor')


train_input = sagemaker.inputs.TrainingInput(
    f's3://aurelia-resort-data/model_train/data_csv/train.csv', 
    content_type='csv'
)


test_input = sagemaker.inputs.TrainingInput(
    f's3://aurelia-resort-data/model_train/data_csv/test.csv', 
    content_type='csv'
)


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [48]:
linear_estimator.fit(inputs={'train': train_input, 'validation': test_input})

INFO:sagemaker:Creating training-job with name: linear-learner-2023-04-04-01-28-15-578


2023-04-04 01:28:21 Starting - Starting the training job......
2023-04-04 01:28:57 Starting - Preparing the instances for training......
2023-04-04 01:30:00 Downloading - Downloading input data...
2023-04-04 01:30:30 Training - Downloading the training image......
2023-04-04 01:31:41 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/04/2023 01:31:57 INFO 139653723461440] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss'

UnexpectedStatusException: Error for Training job linear-learner-2023-04-04-01-28-15-578: Failed. Reason: ClientError: No iterator has been registered for ContentType ('csv', '1.0'), exit code: 2